# Udqaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [27]:
import os
import json

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from lib.evaluation import EvaluationResult

from lib.memory import LongTermMemory
from lib.state_machine import StateMachine
from lib.agents import AgentState

from tavily import TavilyClient

import chromadb
from dotenv import load_dotenv

In [28]:
# Load environment variables
load_dotenv()

True

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [29]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

In [30]:
chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

In [31]:
@tool(
    name="retrieve_game",
    description="""
Semantic search: Finds most results in the vector DB.
Args:
- query: a question about game industry.

You'll receive results as list. Each element contains:
- Platform: like Game Boy, Playstation 5, Xbox 360...
- Name: Name of the Game
- YearOfRelease: Year when that game was released for that platform
- Description: Additional details about the game
"""
)
def retrieve_game(query: str) -> list[dict]:
    results = collection.query(query_texts=[query], n_results=5)
    docs = results.get("documents", [[]])[0]
    metadatas = results.get("metadatas", [[]])[0]
    output = []
    for doc, meta in zip(docs, metadatas):
        output.append({
            "Platform": meta.get("Platform"),
            "Name": meta.get("Name"),
            "YearOfRelease": meta.get("YearOfRelease"),
            "Description": meta.get("Description", doc)
        })
    return output

#### Evaluate Retrieval Tool

In [32]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

In [33]:
llm = LLM()

@tool(
    name="evaluate_retrieval",
    description="""
Based on the user's question and on the list of retrieved documents, 
it will analyze the usability of the documents to respond to that question.
Args:
- question: original question from user
- retrieved_docs: retrieved documents most similar to the user query in the Vector Database

The result includes:
- useful: whether the documents are useful to answer the question
- description: description about the evaluation result
"""
)
def evaluate_retrieval(question: str, retrieved_docs: list[dict]) -> dict:
    prompt = (
        "Your task is to evaluate if the documents are enough to respond the query.\n"
        "Give a detailed explanation, so it's possible to take an action to accept it or not.\n"
        f"Question: {question}\n"
        f"Retrieved Documents: {json.dumps(retrieved_docs, indent=2)}"
    )
    ai_message = llm.invoke(prompt)
    # Parse the result using EvaluationResult (assumes ai_message.content is JSON)
    try:
        result = EvaluationResult.model_validate_json(ai_message.content)
        return {
            "useful": result.task_completion.task_completed,
            "description": result.feedback
        }
    except Exception:
        # fallback: just return the raw content
        return {
            "useful": False,
            "description": ai_message.content
        }

#### Game Web Search Tool

In [34]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

In [35]:
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

@tool(
    name="game_web_search",
    description="""
Semantic search: Finds most results in the vector DB.
Args:
- question: a question about game industry.
"""
)
def game_web_search(question: str) -> list[dict]:
    """
    Uses Tavily client to search the web for game industry questions.
    Args:
        question (str): a question about game industry.
    Returns:
        List[dict]: List of relevant web search results.
    """
    results = tavily_client.search(query=question, max_results=5)
    # Each result contains title, url, snippet, etc.
    return [
        {
            "title": r.get("title"),
            "url": r.get("url"),
            "snippet": r.get("snippet")
        }
        for r in results.get("results", [])
    ]

### Agent

In [36]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

In [37]:
# Create the Agent with StateMachine
instructions = """
You are UdaPlay, an AI Research Agent specialized in the video game industry. Your role is to provide accurate, comprehensive information about video games, gaming platforms, release dates, and industry trends.

Your workflow:
1. First, search your internal knowledge base using the retrieve_game tool for any gaming-related questions
2. Evaluate the retrieved information using the evaluate_retrieval tool to determine if it's sufficient
3. If the internal knowledge is insufficient, use the game_web_search tool to find additional information
4. Provide clear, accurate answers based on the available information
5. Always cite your sources and be transparent about the limitations of your knowledge

Guidelines:
- Be precise with dates, platform names, and game titles
- If information conflicts between sources, mention the discrepancy
- For questions about recent games or industry news, prioritize web search results
- Structure your responses clearly and provide context when helpful
- If you cannot find reliable information, state this clearly rather than guessing
"""

# Initialize the tools
tools = [retrieve_game, evaluate_retrieval, game_web_search]

# Create the agent
udaplay_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=instructions,
    tools=tools,
    temperature=0.3  # Lower temperature for more factual responses
)

print("UdaPlay Agent created successfully!")
print(f"Available tools: {[tool.name for tool in tools]}")

UdaPlay Agent created successfully!
Available tools: ['retrieve_game', 'evaluate_retrieval', 'game_web_search']


In [38]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

In [40]:
# Test queries for the agent
test_queries = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?", 
    "Was Mortal Kombat X released for PlayStation 5?"
]

# Function to test the agent with proper error handling
def test_agent_query(query, session_id="test_session"):
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print(f"{'='*60}")
    
    try:
        # Invoke the agent
        run_result = udaplay_agent.invoke(query, session_id=session_id)
        
        # Get the final state and extract the last AI message
        final_state = run_result.get_final_state()
        if final_state and final_state.get("messages"):
            # Find the last AI message
            for message in reversed(final_state["messages"]):
                if hasattr(message, 'role') and message.role == 'assistant' and message.content:
                    print(f"RESPONSE:\n{message.content}")
                    break
            
            # Print token usage if available
            total_tokens = final_state.get("total_tokens", 0)
            if total_tokens > 0:
                print(f"\nTokens used: {total_tokens}")
        else:
            print("No response generated")
            
    except Exception as e:
        print(f"Error: {str(e)}")
    
    print(f"{'='*60}\n")

# Test each query
for i, query in enumerate(test_queries, 1):
    print(f"Testing Query {i}/{len(test_queries)}")
    test_agent_query(query, session_id=f"test_session_{i}")

Testing Query 1/3

QUERY: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
RESPONSE:
Pokémon Gold and Silver were released in 1999 for the Game Boy Color. These games are notable for being the second generation of Pokémon games, introducing new regions, Pokémon, and gameplay mechanics.

Tokens used: 2674

Testing Query 2/3

QUERY: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] E

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes